# Local Hyper Parameter Search with Optuna


- In this notebook, we'll use Optuna for automatic hyperparameter search to optimize our reservoir parameters for a machine learning task.
- We will go trough the topics of defining an objective function, creating an Optuna study, and launching the optimization process to find the best hyperparameters.
- Parallelization concepts will be shown in the next tutorials but rely on this simple sequential search.

### Step 1: Prepare your data 

In [11]:
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir
from reservoirpy.datasets import doublescroll
from reservoirpy.observables import nrmse

In [12]:
timesteps = 2000
x0 = [0.37926545, 0.058339, -0.08167691]
X = doublescroll(timesteps, x0=x0, method="RK23")

train_len = 1000

X_train = X[:train_len]
y_train = X[1 : train_len + 1]

X_test = X[train_len : -1]
y_test = X[train_len + 1:]

dataset = ((X_train, y_train), (X_test, y_test))

### Step 2: Define fixed parameters for the hyper parameter search

In [13]:
import optuna
from optuna.storages import JournalStorage, JournalFileStorage

optuna.logging.set_verbosity(optuna.logging.ERROR)

- Explain that optimization is going to be done w different trials where a simulation will be done with a configuration of hyperparameters
- Then the configuration is evaluated and stored in a database (can use different ones such as SQL or JournalStorage that we will use there)
- First, define the fixed the with the values u want to use (here units, iss and ridge for the reservoir part, and nb_seeds for the trials of the hyper parameter search)

- In this section of the tutorial, we will conduct an Optuna hyperparameter search through multiple trials. 
- Each trial will involve simulating a model with a specific configuration of hyperparameters. 
- The results of each simulation will be evaluated and stored in a database, which can be implemented using SQL or JournalStorage. 
- Before starting the optimization process, we will define the fixed hyperparameter values we want to use, such as units, iss, and ridge for the reservoir part, and the number of seeds (nb_seeds) for the hyperparameter search trials.

In [4]:
# Trial Fixed hyper-parameters
nb_seeds = 3
N = 500
iss = 0.9
ridge = 1e-7

### Step 3: Define an Objective Function

- In Optuna, an objective function is a user-defined function that evaluates the performance of a model using a specific set of hyperparameters. 
- The main goal of the objective function is to determine the model's performance metric, which Optuna aims to either maximize or minimize during the hyperparameter search.
- The objective function serves as a bridge between the optimization process and the model evaluation, allowing Optuna to iteratively explore different hyperparameter configurations to find the optimal combination that maximizes or minimizes the performance metric.
- It isn't shown here but optuna also supports [multi-objective optimizations](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html).




In [5]:
def objective(trial):

    # Record objective value(s) for each trial
    losses = []

    # Trial generated parameters (with log scale)
    sr = trial.suggest_float("sr_1", 1e-2, 10, log=True)
    lr = trial.suggest_float("lr_1", 1e-3, 1, log=True)

    for seed in range(nb_seeds):
        reservoir = Reservoir(N,
                              sr=sr,
                              lr=lr,
                              input_scaling=iss,
                              seed=seed)
        
        readout = Ridge(ridge=ridge)

        model = reservoir >> readout

        # Train and test your model
        predictions = model.fit(X_train, y_train).run(X_test)

        # Compute the wanted metrics
        loss = nrmse(y_test, predictions, norm_value=np.ptp(X_train))

        losses.append(loss)

    return np.mean(losses)

### Step 4: Create an Optuna Study

- To create an Optuna study, use optuna.create_study()
- Specify a sampling method (RandomSampler(), TPESampler() ...) for exploring hyperparameter space. 
- Choose a storage backend (SQLite, JournalStorage ...) to store trial information. 
- Then, Optuna manages the hyperparameter search process, enabling efficient exploration and optimization.

In [ ]:
# Define study parameters
nb_trials = 200

sampler = optuna.samplers.RandomSampler() 

study_name = 'optuna_tutorial'
log_name = f"optuna-journal_{study_name}.log"

storage = JournalStorage(JournalFileStorage(log_name))

In [ ]:
# Create study
study = optuna.create_study(
    study_name=study_name,
    direction="minimize",
    sampler=sampler,
    storage=storage,
    load_if_exists=True)


### Step 5: Launch Optimization

This might take some time... You can skip this step and directly see the results in the next step.

In [19]:
# Optimize study with Objective function
study.optimize(objective, n_trials=nb_trials)

[I 2023-06-30 18:23:50,046] Trial 0 finished with value: 0.0010597792272651022 and parameters: {'sr_1': 0.01762372195431926, 'lr_1': 0.2698487689223331}. Best is trial 0 with value: 0.0010597792272651022.
[I 2023-06-30 18:23:51,737] Trial 1 finished with value: 0.6217617739366391 and parameters: {'sr_1': 0.8302894390115056, 'lr_1': 0.007075889175862396}. Best is trial 0 with value: 0.0010597792272651022.
[I 2023-06-30 18:23:53,440] Trial 2 finished with value: 0.17629718033442113 and parameters: {'sr_1': 5.68036910293409, 'lr_1': 0.3829624967321427}. Best is trial 0 with value: 0.0010597792272651022.
[I 2023-06-30 18:23:55,223] Trial 3 finished with value: 0.032747301724544126 and parameters: {'sr_1': 0.12231039183478166, 'lr_1': 0.002988784239330389}. Best is trial 0 with value: 0.0010597792272651022.
[I 2023-06-30 18:23:56,904] Trial 4 finished with value: 0.13492908986476793 and parameters: {'sr_1': 0.39219554360250947, 'lr_1': 0.005586911207642662}. Best is trial 0 with value: 0.00

### Step 6: Visualize the results 

- Optuna provides intuitive [visualizations](https://optuna.readthedocs.io/en/stable/reference/visualization/index.html) for hyperparameter optimization, allowing easy analysis and understanding of the search process and results.

In [27]:
from optuna.visualization import plot_slice
from optuna.visualization import plot_contour

In [28]:
plot_slice(study)

In [29]:
plot_contour(study)